In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Device

In [2]:
# Get gpu device

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.backends.cudnn.benchmark = True
torch.cuda.get_device_properties(0)

_CudaDeviceProperties(name='NVIDIA GeForce GTX 1080 Ti', major=6, minor=1, total_memory=11263MB, multi_processor_count=28)

## Download CIFAR100 dataset

In [3]:
# Normalize a tensor image with mean and standard deviation of 0.5
transform = Compose([ToTensor(), Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]).to(device)])

# Download train data
training_data = datasets.CIFAR100(
    root="data",
    train=True,
    download=True,
    transform=transform,
)

# Download test data
test_data = datasets.CIFAR100(
    root="data",
    train=False,
    download=True,
    transform=transform,
)

Files already downloaded and verified
Files already downloaded and verified


### Number of classes

In [4]:
n_classes = len(test_data.classes)
n_classes

100

## Dataloaders

In [5]:
batch_size = 512
image_size = 32 * 32 * 3

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([512, 3, 32, 32])
Shape of y: torch.Size([512]) torch.int64


In [6]:
print(len(train_dataloader.dataset))
print(len(test_dataloader.dataset))

50000
10000


## Define model

In [7]:

print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self, hidden_layers=[512, 256, 128], activations=[nn.ReLU(), nn.ReLU(), nn.ReLU()]):
        super().__init__()
    
        self.flatten = nn.Flatten()

        layers = []

        layers.append(nn.Linear(image_size, hidden_layers[0]))
        layers.append(nn.BatchNorm1d(hidden_layers[0]))
        layers.append(activations[0])
        
        for i in range(len(hidden_layers)-1):
            layers.append(nn.Linear(hidden_layers[i], hidden_layers[i+1]))
            layers.append(nn.BatchNorm1d(hidden_layers[i+1]))
            layers.append(activations[i+1])
            
        layers.append(nn.Linear(hidden_layers[-1], n_classes))
        
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers(x)
        return logits

model = NeuralNetwork(hidden_layers = [1024, 512, 256, 128], activations=[nn.ReLU(), nn.ReLU(), nn.ReLU(), nn.ReLU()]).to(device)
print(model)

Using cuda:0 device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=256, out_features=128, bias=True)
    (10): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Linear(in_features=128, out_features=100, bias=True)
  )
)


## Define optimizer and loss function

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

### Learning rate schedule

In [9]:
import torch.optim.lr_scheduler as lr_scheduler
 
# scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1, end_factor=1e-3, total_iters=10)

## Training loop

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
    # scheduler.step()

In [11]:
def test(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
   
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n lr: {optimizer.param_groups[0]['lr']}")
    print("_______________________________")

In [12]:
epochs = 15

for i in range(epochs):
    print(f"Epoch {i+1}\n_______________________________")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
_______________________________
loss: 4.723909  [  512/50000]
Test Error: 
 Accuracy: 9.6%, Avg loss: 3.921370 
 lr: 0.1
_______________________________
Epoch 2
_______________________________
loss: 3.875136  [  512/50000]
Test Error: 
 Accuracy: 13.6%, Avg loss: 3.702246 
 lr: 0.1
_______________________________
Epoch 3
_______________________________
loss: 3.582415  [  512/50000]
Test Error: 
 Accuracy: 15.2%, Avg loss: 3.581087 
 lr: 0.1
_______________________________
Epoch 4
_______________________________
loss: 3.448618  [  512/50000]
Test Error: 
 Accuracy: 17.2%, Avg loss: 3.488589 
 lr: 0.1
_______________________________
Epoch 5
_______________________________
loss: 3.425128  [  512/50000]
Test Error: 
 Accuracy: 18.0%, Avg loss: 3.423834 
 lr: 0.1
_______________________________
Epoch 6
_______________________________
loss: 3.208062  [  512/50000]
Test Error: 
 Accuracy: 19.5%, Avg loss: 3.394938 
 lr: 0.1
_______________________________
Epoch 7
_____________________